In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.lang.state.*

GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
val model = Network("model")

// Block 1: nodes
val node1 = Delay(model, "Delay")
val node2 = Queue(model, "Queue1", SchedStrategy.FCFS)
val node3 = Queue(model, "Queue2", SchedStrategy.FCFS)
node2.setNumberOfServers(3)
node3.setNumberOfServers(3)

// Block 2: classes
val jobclass1 = ClosedClass(model, "Class1", 2, node1, 0)
val jobclass2 = ClosedClass(model, "Class2", 2, node1, 0)
val jobclass3 = ClosedClass(model, "Class3", 2, node1, 0)
val jobclass4 = ClosedClass(model, "Class4", 1, node1, 0)

node1.setService(jobclass1, Exp(1.0)) // (Delay,Class1)
node1.setService(jobclass2, Exp(1.0)) // (Delay,Class2)
node1.setService(jobclass3, Exp(10.0)) // (Delay,Class3)
node1.setService(jobclass4, Exp(1.0)) // (Delay,Class4)

node2.setService(jobclass1, Exp(1.0)) // (Queue1,Class1)
node2.setService(jobclass2, Erlang(1.0, 2)) // (Queue1,Class2)
node2.setService(jobclass3, Exp(10.0)) // (Queue1,Class3)
node2.setService(jobclass4, Exp(1.0)) // (Queue1,Class4)

node3.setService(jobclass1, Disabled()) // (Queue2,Class1)
node3.setService(jobclass2, Disabled()) // (Queue2,Class2)
node3.setService(jobclass3, Erlang(1.0, 2)) // (Queue2,Class3)
node3.setService(jobclass4, Exp(1.0)) // (Queue2,Class4)

In [ ]:
// Block 3: topology
val routingMatrix = model.initRoutingMatrix()

routingMatrix.set(jobclass1, jobclass1, node1, node2, 0.50)
routingMatrix.set(jobclass1, jobclass1, node2, node1, 1.00)
routingMatrix.set(jobclass1, jobclass1, node3, node1, 1.00)

routingMatrix.set(jobclass1, jobclass2, node1, node2, 0.50)

routingMatrix.set(jobclass2, jobclass2, node2, node1, 1.00)
routingMatrix.set(jobclass2, jobclass2, node3, node1, 1.00)

routingMatrix.set(jobclass2, jobclass1, node1, node2, 1.00)

routingMatrix.set(jobclass3, jobclass3, node1, node2, 0.25)
routingMatrix.set(jobclass3, jobclass3, node1, node3, 0.25)
routingMatrix.set(jobclass3, jobclass3, node2, node1, 1.00)
routingMatrix.set(jobclass3, jobclass3, node3, node1, 1.00)

routingMatrix.set(jobclass3, jobclass4, node1, node2, 0.50)

routingMatrix.set(jobclass4, jobclass4, node2, node1, 1.00)
routingMatrix.set(jobclass4, jobclass4, node3, node1, 1.00)

routingMatrix.set(jobclass4, jobclass3, node1, node2, 1.00)

model.link(routingMatrix)

// Create nodes array for State operations
val node = arrayOf(node1, node2, node3)

In [ ]:
// State operations like MATLAB - using node index 1 (second node, node2)
val spaceRunning = State.fromMarginalAndRunning(model, 1, intArrayOf(2,1,1,1), intArrayOf(2,1,0,0))
val spaceStarted = State.fromMarginalAndStarted(model, 1, intArrayOf(2,1,1,1), intArrayOf(2,1,0,0))
val space = State.fromMarginal(model, 1, intArrayOf(2,1,1,1))

// This example demonstrates State operations as shown in MATLAB
println("spaceRunning: $spaceRunning")
println("spaceStarted: $spaceStarted")
println("space: $space")